#### All you have to prepare for dataset of CellClass is a dataframe with following information and create mask from outlines which is project specific:
* Metadata_Plate     -          object
* Metadata_Well       -         object
* Metadata_Site        -         int64
* Metadata_Location2   -        object
* Metadata_Location     -       object
* ObjectNumber          -        int64
* ClustersForGFP        -        int64
* Nuclei_Location_Center_X -   float64
* Nuclei_Location_Center_Y -   float64
* URL_OrigDNA             -     object
* URL_OrigER             -      object
* URL_OrigMito           -      object
* URL_OrigProtein        -      object
* P-W                    -      object
* P-W-S                  -      object
* Metadata_Location_2    -      object
* **bbox**                   -      object
* **image_id**               -       int64
* **cat_id**                 -       int64
* **im_paths**               -      object
* height                 -       int64
* width                  -       int64
* iscrowd                -       int64
* mask                   -      object
* subset_label           -      object
    

In [6]:
import pandas as pd
import numpy as np
import skimage

###### Sample Input dataframe + few project specific parameters

In [2]:
import pandas as pd
#### directories
dataset_dir='/storage/data/marziehhaghighi/rareDisease/'
model_dir = "/storage/data/marziehhaghighi/rareDisease/deepLearningModels/MaskRCNN/"

#### all data information in one dataframe
# annotDate='20200208'
annotDate='20200208-scMarziehModif'
dfInfo = pd.read_csv(dataset_dir+'PILOT_1_maxproj/df-Info-'+annotDate+'.csv')
project_name="rare_disease_classification"
dfInfo.head()

,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Location2,Metadata_Location,ObjectNumber,ClustersForGFP,Nuclei_Location_Center_X,Nuclei_Location_Center_Y,URL_OrigDNA,...,Metadata_Location_2,bbox,image_id,cat_id,im_paths,height,width,iscrowd,mask,subset_label
0,RC4_IF_43,G04,2,cytoplasm,cytoplasm,12,1,668.219256,322.023299,/storage/data/marziehhaghighi/rareDisease/PILO...,...,ne,"[570, 232, 203, 206]",23,4,['/storage/data/marziehhaghighi/rareDisease/PI...,1080,1080,0,/storage/data/marziehhaghighi/rareDisease/PILO...,train
1,RC4_IF_43,G04,2,perinuclear,perinuclear,33,1,284.860093,863.067541,/storage/data/marziehhaghighi/rareDisease/PILO...,...,ne,"[189, 730, 196, 207]",23,16,['/storage/data/marziehhaghighi/rareDisease/PI...,1080,1080,0,/storage/data/marziehhaghighi/rareDisease/PILO...,train
2,RC4_IF_43,G04,2,not expressed,not expressed,2,0,317.139393,111.299443,/storage/data/marziehhaghighi/rareDisease/PILO...,...,ne,"[243, 35, 153, 164]",23,10,['/storage/data/marziehhaghighi/rareDisease/PI...,1080,1080,0,/storage/data/marziehhaghighi/rareDisease/PILO...,train
3,RC4_IF_43,G04,2,not expressed,not expressed,3,0,189.935506,143.718273,/storage/data/marziehhaghighi/rareDisease/PILO...,...,ne,"[106, 52, 151, 196]",23,10,['/storage/data/marziehhaghighi/rareDisease/PI...,1080,1080,0,/storage/data/marziehhaghighi/rareDisease/PILO...,train
4,RC4_IF_43,G04,2,not expressed,not expressed,4,-1,554.033642,142.660727,/storage/data/marziehhaghighi/rareDisease/PILO...,...,ne,"[479, 47, 149, 225]",23,10,['/storage/data/marziehhaghighi/rareDisease/PI...,1080,1080,0,/storage/data/marziehhaghighi/rareDisease/PILO...,train


In [9]:
# dfInfo['im_paths'][0]

In [8]:
# dfInfo.dtypes

## How to prepare the above dataframe based on single cell profiles
* Note that for this specific project we have Metadata_Location which is the label for each single cell, this may be different for each project, you have to change it to Metadata_Label

In [3]:
### here is a subset of single cell profiles with 
df_SC = pd.read_csv('/storage/data/marziehhaghighi/rareDisease/PILOT_1_maxproj/scLebelsImgs-20200208-scMarziehModif.csv')
df_SC['Metadata_Label']=df_SC['Metadata_Location2']
imURLS=['URL_OrigProtein','URL_OrigER','URL_OrigMito','URL_OrigDNA']
originalURLpath='file:/home/ubuntu/bucket/projects/2017_09_27_RareDiseases_Taipale'
toBeReplaced='/storage/data/marziehhaghighi/rareDisease'

df_SC.head()

,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Location2,Metadata_Location,ObjectNumber,ClustersForGFP,Nuclei_Location_Center_X,Nuclei_Location_Center_Y,URL_OrigDNA,URL_OrigER,URL_OrigMito,URL_OrigProtein,Metadata_Label
0,RC4_IF_43,G04,2,cytoplasm,perinuclear foci,12,1,668.219256,322.023299,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,cytoplasm
1,RC4_IF_43,G04,2,perinuclear,perinuclear foci,33,1,284.860093,863.067541,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,perinuclear
2,RC4_IF_43,G04,2,not expressed,not expressed,2,0,317.139393,111.299443,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,not expressed
3,RC4_IF_43,G04,2,not expressed,not expressed,3,0,189.935506,143.718273,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,not expressed
4,RC4_IF_43,G04,2,not expressed,not expressed,4,-1,554.033642,142.660727,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,file:/home/ubuntu/bucket/projects/2017_09_27_R...,not expressed


In [4]:
from sklearn import preprocessing
df_SC['P-W']=df_SC['Metadata_Plate']+'-'+df_SC['Metadata_Well']
df_SC['P-W-S']=df_SC['Metadata_Plate']+'-'+df_SC['Metadata_Well']+'-'+df_SC['Metadata_Site'].astype(str)

dfInfo=df_SC.copy()
# dfInfo=df_SC[~df_SC['Metadata_Location2'].isin(['na','?'])]
# dfInfo['Metadata_Location']=dfInfo['Metadata_Location2']
dfInfo['bbox']=""
dfInfo['bbox']=dfInfo['bbox'].astype(object)
le_imageIds = preprocessing.LabelEncoder()
le_imageIds.fit(dfInfo['URL_OrigProtein'])
dfInfo['image_id']=le_imageIds.transform(dfInfo[imURLS[0]])
dfInfo['image_id']=dfInfo['image_id']+1
# list(le_imageIds.classes_)
le_catIds = preprocessing.LabelEncoder()
le_catIds.fit(dfInfo['Metadata_Label'])
dfInfo['cat_id']=le_catIds.transform(dfInfo['Metadata_Label'])
dfInfo['cat_id']=dfInfo['cat_id']+1

# originalURLpath='file:/home/ubuntu/bucket/projects/2017_09_27_RareDiseases_Taipale'
# toBeReplaced='/storage/data/marziehhaghighi/rareDisease'
# imURLS=['URL_OrigProtein','URL_OrigER','URL_OrigMito','URL_OrigDNA']
for s in imURLS:
    dfInfo[s]=dfInfo[s].str.replace(originalURLpath,toBeReplaced)

    
make_union_list = lambda c1, c2: [c1, c2]
add_to_list = lambda c1, c2: [c2]+c1
dfInfo['im_paths']=dfInfo[imURLS[0]].combine(dfInfo[imURLS[1]], make_union_list)\
.combine(dfInfo[imURLS[2]], add_to_list)\
.combine(dfInfo[imURLS[3]], add_to_list)

dfInfo.head()


,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Location2,Metadata_Location,ObjectNumber,ClustersForGFP,Nuclei_Location_Center_X,Nuclei_Location_Center_Y,URL_OrigDNA,URL_OrigER,URL_OrigMito,URL_OrigProtein,Metadata_Label,P-W,P-W-S,bbox,image_id,cat_id,im_paths
0,RC4_IF_43,G04,2,cytoplasm,perinuclear foci,12,1,668.219256,322.023299,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,cytoplasm,RC4_IF_43-G04,RC4_IF_43-G04-2,,56,5,[/storage/data/marziehhaghighi/rareDisease/PIL...
1,RC4_IF_43,G04,2,perinuclear,perinuclear foci,33,1,284.860093,863.067541,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,perinuclear,RC4_IF_43-G04,RC4_IF_43-G04-2,,56,18,[/storage/data/marziehhaghighi/rareDisease/PIL...
2,RC4_IF_43,G04,2,not expressed,not expressed,2,0,317.139393,111.299443,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,not expressed,RC4_IF_43-G04,RC4_IF_43-G04-2,,56,12,[/storage/data/marziehhaghighi/rareDisease/PIL...
3,RC4_IF_43,G04,2,not expressed,not expressed,3,0,189.935506,143.718273,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,not expressed,RC4_IF_43-G04,RC4_IF_43-G04-2,,56,12,[/storage/data/marziehhaghighi/rareDisease/PIL...
4,RC4_IF_43,G04,2,not expressed,not expressed,4,-1,554.033642,142.660727,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,/storage/data/marziehhaghighi/rareDisease/PILO...,not expressed,RC4_IF_43-G04,RC4_IF_43-G04-2,,56,12,[/storage/data/marziehhaghighi/rareDisease/PIL...


In [16]:
# dfInfo['Metadata_Label'].unique()

In [10]:
masksSaveFolder=dataset_dir+'PILOT_1_maxproj/masks-'+annotDate+'/'
from PIL import Image
import skimage.io
from skimage.segmentation import flood_fill


imagesPWS=dfInfo['P-W-S'].unique().tolist()

for im in imagesPWS:
    imDf=dfInfo[dfInfo['P-W-S']==im].reset_index(drop=True);
    
    oulineImageFile=dataset_dir+'/PILOT_1_maxproj/Cell_outlines/'+imDf.Metadata_Plate[0]+'-'+imDf.Metadata_Well[0]+\
    '/Cell_outlines/'+imDf.Metadata_Well[0]+'_s'+str(imDf.Metadata_Site[0])+'_cell_outlines.png'
    oulineImage=Image.open(oulineImageFile)
    oulineImage1=np.asarray(oulineImage.convert('RGB'))[:,:,1].copy()
    edgePixValue=(3/4)*oulineImage1.max()
    
#     plt.figure()
#     plt.hist(oulineImage1.flatten(), bins=20)
#     plt.imshow(oulineImage1)
#     plt.show()       
    oulineImage1[oulineImage1 < edgePixValue] = 0    # Black
    oulineImage1[oulineImage1 >= edgePixValue] = 255
    maskArrayOneHot=np.zeros((oulineImage1.shape[0],oulineImage1.shape[1],imDf.shape[0]))
    maskArray=np.zeros((oulineImage1.shape[0],oulineImage1.shape[1]),dtype='uint8')
    for i in imDf.index:
        cellDf=imDf.loc[i,:]
        cell_cent_X,cell_cent_Y=int(cellDf['Nuclei_Location_Center_X']),int(cellDf['Nuclei_Location_Center_Y'])

        cellMask1 = skimage.segmentation.flood_fill(oulineImage1, (cell_cent_Y,cell_cent_X), 200, tolerance=10)
        cellMask1[cellMask1 != 200] = 0    
        cellMask = skimage.morphology.remove_small_holes(cellMask1.astype(int), area_threshold=200)
        maskArrayOneHot[:,:,i]=cellMask
        minc,maxc=np.where(cellMask==True)[1].min(),np.where(cellMask==True)[1].max()+1
        minr,maxr=np.where(cellMask==True)[0].min(),np.where(cellMask==True)[0].max()+1
#         roi=[minr, minc, maxr, maxc]
        bbox=[minc, minr,maxc - minc, maxr - minr]
        dfInfInd=dfInfo.loc[(dfInfo['P-W-S']==im) & (dfInfo['ObjectNumber']==cellDf['ObjectNumber'])].index[0]
        dfInfo.at[dfInfInd,'bbox']=bbox
        
        cellMask1=cellMask.astype(int).copy()
        cellMask1[cellMask1!=0]=cellDf['ObjectNumber'];#(i+1)*1;
                       
        maskArray=maskArray+cellMask1
        
    maskFileName=masksSaveFolder+im+'_cell_mask.png'   
###     maskFileName=masksSaveFolder+imDf.Metadata_Well[0]+'_s'+str(imDf.Metadata_Site[0])+'_cell_mask.png'
#     skimage.io.imsave(maskFileName,maskArray.astype('uint8'))

    if 0:
        plt.figure(figsize=(4,4))
        plt.imshow(maskArray)
        plt.show()   
        
dfInfo['height']=oulineImage1.shape[0]
dfInfo['width']=oulineImage1.shape[1]
dfInfo['iscrowd']=0
dfInfo['mask']=masksSaveFolder+dfInfo['P-W-S']+'_cell_mask.png'

# dfInfo.to_csv(dataset_dir+'PILOT_1_maxproj/df-Info-'+annotDate+'-test.csv', index=False)

/home/marziehhaghighi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?


In [11]:
cell_cent_X,cell_cent_Y

(928, 950)

In [12]:
bbox

[832, 883, 175, 161]

In [13]:
832+175

1007

In [ ]:
#         mask_root_dir= '/raid/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/workspace/analysis/2016_04_01_a549_48hr_batch1/'
#         maskFname=mask_root_dir+plateName+'/'+'/analysis/'+well+'-'+site+'/outlines/'+plateName+'/'+well+'_s'+site+'--cell_outlines.png'
# #         print(maskFname)
#         maskIm=skimage.io.imread(maskFname)
#         from skimage.segmentation import flood_fill
#         cellMask1 = skimage.segmentation.flood_fill(maskIm, (yCenterM,xCenterM), 1)
#         cellMask1[cellMask1 != 1] = 0  
#         cellMask2=skimage.transform.resize(cellMask1, [compressed_im_size,compressed_im_size], mode="constant",\
#                                  anti_aliasing=True,preserve_range=True)
        
#         cellMask2[cellMask2 != 1] = 0 
#         cellMask3=cellMask2[yCenter-hbs:yCenter+hbs,xCenter-hbs:xCenter+hbs]

In [31]:
maskArray.shape

(1080, 1080)

In [35]:
# dfInfo['ObjectNumber'].min()

1

In [1]:
dfInfo

NameError: name 'dfInfo' is not defined